In [11]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [12]:
dataset = [['Milk', 'Cookie', 'Bread', 'Jam','Ham'],
['Ramen', 'Cookie', 'Milk', 'Kimchi', 'Jam'], 
['Cookie', 'Bear', 'Kimchi', 'Ham', 'Cola', 'Snack'],
['Milk', 'Bread', 'Rice', 'Cheese', 'Cookie', 'Cola'],
['Cookie', 'Snack', 'Kimchi', 'Bear', 'Ramen','Bread'],
['Ramen' 'Cookie', 'Kimchi','Cheese'],
['Rice','Snack','Cookie','Jam','Milk'],
['Rice','Kimchi','Bread','Bear','Cheese','Ham'],
['Kimchi','Kimchi','Cookie','Snack','Cola','Ham'],           
['Ham','Ramen','Cheese','Rice','Snack','Bear','Jam']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Bear,Bread,Cheese,Cola,Cookie,Ham,Jam,Kimchi,Milk,Ramen,RamenCookie,Rice,Snack
0,False,True,False,False,True,True,True,False,True,False,False,False,False
1,False,False,False,False,True,False,True,True,True,True,False,False,False
2,True,False,False,True,True,True,False,True,False,False,False,False,True
3,False,True,True,True,True,False,False,False,True,False,False,True,False
4,True,True,False,False,True,False,False,True,False,True,False,False,True
5,False,False,True,False,False,False,False,True,False,False,True,False,False
6,False,False,False,False,True,False,True,False,True,False,False,True,True
7,True,True,True,False,False,True,False,True,False,False,False,True,False
8,False,False,False,True,True,True,False,True,False,False,False,False,True
9,True,False,True,False,False,True,True,False,False,True,False,True,True


In [15]:
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

# 지지도를 기준으로 내림차순
frequent_itemsets
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
3,0.7,(Cookie)
6,0.6,(Kimchi)
4,0.5,(Ham)
9,0.5,(Snack)
0,0.4,(Bear)
1,0.4,(Bread)
2,0.4,(Cheese)
5,0.4,(Jam)
7,0.4,(Milk)
8,0.4,(Rice)


In [21]:
#itemsets의 제품 개수에 따른 필터링을 위해 length열을 추가
#결과를 frequent_itemsets라는 이름으로 생성
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x:len(x))
frequent_itemsets

,support,itemsets,length
0,0.4,(Bear),1
1,0.4,(Bread),1
2,0.4,(Cheese),1
3,0.7,(Cookie),1
4,0.5,(Ham),1
5,0.4,(Jam),1
6,0.6,(Kimchi),1
7,0.4,(Milk),1
8,0.4,(Rice),1
9,0.5,(Snack),1


In [22]:
rules1=association_rules(frequent_itemsets, metric='lift', min_threshold=0.8)
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cookie),(Kimchi),0.7,0.6,0.4,0.571429,0.952381,-0.02,0.933333,-0.142857
1,(Kimchi),(Cookie),0.6,0.7,0.4,0.666667,0.952381,-0.02,0.900000,-0.111111
2,(Cookie),(Milk),0.7,0.4,0.4,0.571429,1.428571,0.12,1.400000,1.000000
3,(Milk),(Cookie),0.4,0.7,0.4,1.000000,1.428571,0.12,inf,0.500000
4,(Cookie),(Snack),0.7,0.5,0.4,0.571429,1.142857,0.05,1.166667,0.416667
5,(Snack),(Cookie),0.5,0.7,0.4,0.800000,1.142857,0.05,1.500000,0.250000


In [23]:
# 향상도가 1.2이상인 패턴을 발견
rules2 = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 1.2)
rules2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cookie),(Milk),0.7,0.4,0.4,0.571429,1.428571,0.12,1.4,1.0
1,(Milk),(Cookie),0.4,0.7,0.4,1.000000,1.428571,0.12,inf,0.5


In [24]:
# frequent_itemsets에서 특정 조건을 만족시키는 행들을 선택
frequent_itemsets1 = frequent_itemsets[(frequent_itemsets['length'] == 2) & (frequent_itemsets['support'] >= 0.4)]
frequent_itemsets1

,support,itemsets,length
10,0.4,"(Cookie, Kimchi)",2
11,0.4,"(Cookie, Milk)",2
12,0.4,"(Cookie, Snack)",2
